## Train Othello-GPT and save to `ckpts`

Use `jupyter nbconvert --execute --to notebook --allow-errors --ExecutePreprocessor.timeout=-1 train_gpt_othello.ipynb --inplace --output ckpts/checkpoint.ipynb` to run in background

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# make deterministic
from mingpt.utils import set_seed
set_seed(44)

In [4]:
import os
import math
import time
from tqdm import tqdm
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn
from torch.nn import functional as F
from data import get_othello
from data.othello import permit, start_hands, OthelloBoardState, permit_reverse
from mingpt.dataset import CharDataset
from mingpt.utils import sample
from mingpt.model import GPT, GPTConfig
from mingpt.trainer import Trainer, TrainerConfig

In [5]:
synthetic_or_championship = True  # True for training on the synthetic dataset

In [6]:
othello = get_othello(ood_num=-1, data_root=None if synthetic_or_championship else "data/othello_championship", wthor=True)
train_dataset = CharDataset(othello)
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, n_layer=8, n_head=8, n_embd=512)
model = GPT(mconf)

  0%|          | 0/50 [00:00<?, ?it/s]

Mem Used: 4.154 GB: 100%|██████████| 50/50 [00:18<00:00,  2.73it/s]


Deduplicating...
Deduplicating finished with 4999389 games left
Using 20 million for training, 999389 for validation
Dataset created has 4000000 sequences, 61 unique words.


In [ ]:
max_epochs = 250
# initialize a trainer instance and kick off training
t_start = time.strftime("_%Y%m%d_%H%M%S")
tconf = TrainerConfig(
    max_epochs=max_epochs, 
    batch_size=512*4, # using 4 gpus
    learning_rate=5e-4,
    lr_decay=True, 
    warmup_tokens=len(train_dataset)*train_dataset.block_size*5, 
    final_tokens=len(train_dataset)*train_dataset.block_size*max_epochs,
    num_workers=0, 
    ckpt_path=f"./ckpts/gpt_at{t_start}.ckpt", 
)
trainer = Trainer(model, train_dataset, None, tconf)
device = trainer.device
print(t_start)
trainer.train()


  0%|          | 0/1954 [00:00<?, ?it/s]

_20230627_203739


/home/jnbaldwin/miniconda3/envs/othello/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:65: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
epoch 1 iter 1953: train loss 2.03086. lr 1.000000e-04: 100%|██████████| 1954/1954 [08:26<00:00,  3.86it/s]
epoch 2 iter 489: train loss 2.03632. lr 1.250880e-04:  25%|██▌       | 490/1954 [02:10<05:40,  4.30it/s]  

## Or load trained model from `ckpts`

In [ ]:
load_res = model.load_state_dict(torch.load("./ckpts/gpt_synthetic.ckpt" if synthetic_or_championship else "./ckpts/gpt_championship.ckpt"))
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    model = model.to(device)

In [7]:
load_res = model.load_state_dict(torch.load("./ckpts/gpt_at_20230627_213451.ckpt"))
device = torch.cuda.current_device()
model = model.to(device)

## Validate it: for what percentage of all partial games in validation set, the top-1 prediction is legal

In [7]:
if not synthetic_or_championship:  # for GPT trained on both datasets, use the validation set of synthetic for validation
    othello = get_othello(ood_num=-1, data_root=None, wthor=True)

In [8]:
total_nodes = 0
success_nodes = 0

bar = tqdm(othello.val[:1000])
for whole_game in bar:
    length_of_whole_game = len(whole_game)
    for length_of_partial_game in range(1, length_of_whole_game):
        total_nodes += 1
        context = whole_game[:length_of_partial_game]
        x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None, ...].to(device)
        y = sample(model, x, 1, temperature=1.0)[0]
        completion = [train_dataset.itos[int(i)] for i in y if i != -1]
        try:
            OthelloBoardState().update(completion, prt=False)
        except Exception:
#             fail_nodes.append([permit_reverse(_) for _ in context])
            pass
        else:
            success_nodes += 1
    bar.set_description(f"{success_nodes/total_nodes*100:.2f}% pass rate: {success_nodes}/{total_nodes} among all searched nodes")
print(f"{success_nodes/total_nodes*100:.2f}% pass rate: {success_nodes}/{total_nodes} among all searched nodes")

99.91% pass rate: 58936/58990 among all searched nodes: 100%|██████████| 1000/1000 [06:50<00:00,  2.43it/s]

99.91% pass rate: 58936/58990 among all searched nodes


In [ ]:
1 - success_nodes/total_nodes